In [14]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [17]:
analyzer = SentimentIntensityAnalyzer()

In [4]:
df = pd.read_csv("../datasets/gender_only.csv") 

In [18]:
# Apply VADER to each tweet
def get_sentiment(text):
    return analyzer.polarity_scores(text)['compound']

df['sentiment_score'] = df['tweet_text'].apply(get_sentiment)

In [5]:
df.head()

,tweet_text,cyberbullying_type
0,rape is real..zvasiyana nema jokes about being...,gender
1,You never saw any celebrity say anything like ...,gender
2,"@ManhattaKnight I mean he's gay, but he uses g...",gender
3,RT @Raul_Novoa16: @AliciaBernardez @Alex_Aim @...,gender
4,Rape is rape. And the fact that I read one pos...,gender


In [ ]:
primary_keywords = ["she", "her", "woman", "women", "girl", "female","females","femcel","feminazi","hoe", 
                    "bitch","bitches","whore","tits","pussy","vag","hag","tramp","Karen","thot","OF","onlyfans","attention seeker"]
secondary_keywords = ["gold digger", "tramp", "cunt", "wife","girlfriend","gyal","feminist","feminists"]

In [12]:
# Auto-tag likely female-targeted tweets
df['is_targeting_women'] = df['tweet_text'].str.lower().apply(
    lambda text: any(word in text for word in primary_keywords)
)

# Flag tweets that might need manual review
df['needs_review'] = df['tweet_text'].str.lower().apply(
    lambda text: any(word in text for word in secondary_keywords)
)

In [11]:
df[df['is_targeting_women'] == True].to_csv('../datasets/tweets_targeting_women.csv', index=False)
df[df['needs_review'] == True].to_csv('../datasets/tweets_needs_review.csv', index=False)

In [13]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [vaderSentiment]m [charset-normalizer]


In [19]:
df['is_hostile_to_women'] = (df['is_targeting_women']) & (df['sentiment_score'] < -0.05)

In [20]:
df['is_hostile_needs_review'] = (df['needs_review']) & (df['sentiment_score'] < -0.05)

In [22]:
df[df['is_hostile_to_women']].to_csv("../datasets/hostile_tweets_targeting_women.csv", index=False)
df[df['is_hostile_needs_review']].to_csv("../datasets/hostile_tweets_needs_review.csv", index=False)